In [ ]:
# Import Standard Packages
import os 
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from math import cos, sin, pi, exp, tan
# Imports to supress compiler warnings which can be ignored
import warnings
from pyopencl import CompilerWarning
warnings.filterwarnings('ignore', category=CompilerWarning)
# Import Simulation Packages & Extras
from spimagine import volshow
import biobeam
from gputools import fft_convolve, perlin3
from tifffile import imsave, imread
from IPython.display import clear_output
import scipy

# Generates a random point on a sphere with radius r
# Input: 
# r - radius of sphere
def randomSpherePoint(r = 1):
    phi = 2*pi*random.random()
    theta = pi*random.random()
    return np.array([cos(phi)*sin(theta),sin(phi)*sin(theta),cos(theta)])*r*random.random()
# Usage: Random displacement of nuclei positions

# Plots a volume using volshow (single call, no additional requirements)
# Input:
# img - 3d image
def plotvol(img):
    %gui qt5
    w = volshow(img) 
    w.set_colormap("hot")
    return w

# Generates the 2D-cross-section of an (angled) cylindrical light sheet.
# Edges are damped accordingly to reduce edge effects.
# Input:
# size   - size of cross-section (2-tuple)
# units  - resolution (μm per pixel) (3-tuple)
# lam    - wavelength of light (in μm)
# theta  - angle of light-sheet from forward direction (in °)
# z_foc  - position of focal plane in forward direction
# NA     - numerical aperture
def angledLightsheet(size = (512,256),units = (.2,)*3, lam = 0.5, theta = 0, z_foc = 0, NA = .1):
     # Transforming angle to radial
    theta = 2*pi*theta/360
    
    [dx,dy,dz] = units
    
    # Calculating wave number in the x-direction
    k0 = 2*pi/lam
    kx = k0*sin(theta)
    
    # Calculating wave number differences
    dkx = 2*pi/(units[0]*size[0])
    dky = 2*pi/(units[1]*size[1])
    
    x,y = np.meshgrid(np.linspace(-size[0]/2,size[0]/2-1,size[0]),np.linspace(-size[1]/2,size[1]/2-1,size[1]))
    
    # Calculating initial field and adjusting propagation direction to be tilted with angle theta
    E_z0 = biobeam.focus_field_cylindrical_plane(shape=(size[0],size[1]),units =  units[0:2],NA = NA, lam = 0.5,z = 0)
    mask = np.multiply(1*(abs(x) > cos(theta)*size[0]/3),np.exp(-0.01*(abs(x)-cos(theta)*size[0]/3))) + 1*(abs(x) <= cos(theta)*size[0]/3)
    E_z0 = np.multiply( E_z0, mask)
    E_z0 = np.multiply( E_z0, np.exp(-1j*kx*dx*x))
    
    # Propagator
    H_dz = -(1j)*np.multiply(np.lib.scimath.sqrt(k0**2-(x*dkx)**2-(y*dky)**2),(dz*z_foc))
    H_dz = np.exp(np.multiply(H_dz,-np.sign(H_dz.real)+1*(H_dz.real == 0)))
    return np.fft.ifft2(np.fft.ifftshift(np.multiply(np.fft.fftshift(np.fft.fft2(E_z0)),H_dz)))

# Generates *reflective index distribution (rid)* and the *fluorescence distribution (fld)*
#for randomly sampled non-overlapping spheres
# Sphere radii are sampled iid from a beta distribution.
# Input:
# size - size of volume (3-tuple)
# n    - targeted number of spheres
# r    - [min. radius, max. radius, beta parameter, glow ring, membrane thickness, separator thikness, nulceus radius]
# p    - probabilities for [scattering, membrane, nucleus]
def SphereGeometry(size = (512,512,512), n = 200, r = [20,40,5,2,3,2,2], p = [0.15,0.2,0.2],distribution='beta'):
    # Initialization
    obj = 0  # number of placed objects
    step = 0 # number of steps
    c = np.zeros((n,3)) # storage for center of spheres/objects
    rc = np.zeros(n)    # storage for radia of spheres
    rid = np.zeros(size,dtype = int) # refractive index distribution (initialization)
    fld = np.zeros(size,dtype = int) # fluorescence distribution (initialization)

    # Here the main function begins:
    while (obj < n) and step <= 3*n:
        step += 1
        # Add a random center
        c[obj,:] = np.array([random.randint(-int(r[0]/2),size[0]+int(r[0]/2)),random.randint(-int(r[0]/2),size[1]+int(r[0]/2)),random.randint(-int(r[0]/2),size[2]+int(r[0]/2))]);
        # Create a radius if none is chosen yet
        if rc[obj] == 0:
            if distribution == 'beta':
                rc[obj] = r[0] + (r[1]-r[0])*np.random.beta(r[2],r[2])
            elif distribution == 'uniform':
                rc[obj] = r[0] + (r[1]-r[0])*np.random.rand()
            else:
                raise ValueError('distribution must be beta or uniform.')
        # Check if center and radius are valid (i.e. no overlap)
        isvalid = True 
        for i in range(obj):
            if np.linalg.norm(c[obj,:]-c[i,:])<rc[obj]+rc[i]:
                isvalid = False
        # Proceed if valid:
        if isvalid:
            isscatter = random.random() < pscatter # is the cell a scatterer?
            hasmembrane = random.random() < pmembrane # has the cell a membrane?
            hasnucleus = random.random() < pnucleus # has the cell a nucleus?
            cnucleus = randomSpherePoint(rc[obj]-rn-rm) # center of the nucleus
            
            # Iterate over bounding box (naive)
            for h in range(max(int(c[obj,0]-rc[obj]-r[3]),0),min(int(c[obj,0]+rc[obj]+r[3]),size[0])):
                for j in range(max(int(c[obj,1]-rc[obj]-r[3]),0),min(int(c[obj,1]+rc[obj]+r[3]),size[1])):
                    for l in range(max(int(c[obj,2]-rc[obj]-r[3]),0),min(int(c[obj,2]+rc[obj]+r[3]),size[2])):
                        dist = np.linalg.norm(c[obj,:]-np.array([h,j,l]))
                        # Glowing ring around the sphere ("glow")
                        if (dist <= rc[obj]+r[3]) and (dist > rc[obj]):
                            fld[h,j,l] = 1 # glow only affect the fluorescence distribution
                        # Membrane of the sphere ("membrane")
                        elif (dist <= rc[obj]) and (dist>rc[obj]-r[4]):
                            fld[h,j,l] = hasmembrane*2 + (1-hasmembrane)*3
                            rid[h,j,l] = hasmembrane*isscatter*2 + (1-hasmembrane)*isscatter*3
                        # Separating "ring" between membrane & cell
                        elif (dist <= rc[obj]) and (dist>rc[obj]-r[4]-r[5]):
                            fld[h,j,l] = 0
                            rid[h,j,l] = 0
                        # Nucleus of the sphere ("nucleus")   
                        elif (np.linalg.norm(c[obj,:]+cnucleus-np.array([h,j,l])) <= r[6]):
                            fld[h,j,l] = hasnucleus*4 + (1-hasnucleus)*3
                            rid[h,j,l] = hasnucleus*isscatter*4 + (1-hasnucleus)*isscatter*3
                        # General sphere / everything else ("general")
                        elif (dist < rc[obj]):
                            fld[h,j,l] = 3
                            rid[h,j,l] = isscatter*3 
            # increase the number of placed spheres/objects                 
            obj += 1 
            clear_output(wait=True)
            print('Current number of spheres: ',str(obj),end='\r')
    # If the number of steps exceed a certain threshold while not being done the algorithm stops.        
    if step == 3*n:
        print('The number of spheres could not be achieved!\n Only ' + str(obj) + ' spheres have been placed.')
    return fld, rid



## Generation of Synthetic Data: Embryo
The synthetic dataset we model in this dataset depicts a multicellular organism with complex refractive index distribution. We combine several several spheres and ellipsoids to achieve a similar look to the one given by Weigert et al. [1]. In their article they state to use refractive indices in the range $n=1.33-1.38$. Liu et al. [2] state several typical refractive indices for various organelles in a cell: 

| Cell Part        | Refractive Index |
| ---------------- | ---------------- |
| **Cytosol**      | $$n=1.36-1.39$$    |
| **Nucleus**      | $$n=1.355-1.365$$  |
| **Nucleolus**    | $$n=1.375-1.385$$  |
| **Mitochondria** | $$n=1.4-1.42$$     |
| **Lysosome**     | $$n=1.6$$          |

We will use these values in our considerations during modeling of the synthetic data.

[1] M. Weigert, K. Subramanian, S. T. Bundschuh, et al., “Biobeam—multiplexed wave-optical
simulations of light-sheet microscopy,” PLoS Comput. Biol. 14, 1–11 (2018).

[2] P. Y. Liu, L. K. Chin, W. Ser, et al., “Cell refractive index for cell biology and disease diagnosis:
past, present and future,” Lab Chip 16, 634–644 (2016).

In [ ]:
%%capture
random.seed(123)
# Image size
size = np.array([256,512,256])
y,x,z = np.meshgrid(np.arange(0,size[1]),np.arange(0,size[0]),np.arange(0,size[2]))

center = size/2
# Ellipsoidal boundary - Parameters
Radius_Glow     = np.array([115,225,115])
Radius_Shell    = np.array([105,215,105])
Radius_Interior = np.array([42,86,42])
# Generation of Masks
OuterGlow_Mask  = ((x-center[0])**2 / Radius_Glow[0]**2 \
                 + (y-center[1])**2 / Radius_Glow[1]**2 \
                 + (z-center[2])**2 / Radius_Glow[2]**2 <= 1)*1
OuterShell_Mask = ((x-center[0])**2 / Radius_Shell[0]**2 \
                 + (y-center[1])**2 / Radius_Shell[1]**2 \
                 + (z-center[2])**2 / Radius_Shell[2]**2 <= 1)*1
InnerShell_Mask = ((x-center[0])**2 / Radius_Interior[0]**2 \
                 + (y-center[1])**2 / Radius_Interior[1]**2 \
                 + (z-center[2])**2 / Radius_Interior[2]**2 <= 1)*1
# Sampling of spheres between outer and inner shell - Rejection Sampling
Geometry_Classes = np.zeros(size)
Radius_Cell      = 8
Membrane_Cell    = 2
Glow_Cell        = 1
Separator_Cell   = 3
Cell_Centers     = np.zeros((5000,3),'int16')
Cell_Numbers     = 0 
Rejected         = 0
while Rejected < 100:
    Cell_Centers[Cell_Numbers,:] = np.array(
                      [random.randint(-Radius_Shell[0]+Radius_Cell+Membrane_Cell+Glow_Cell,
                                       Radius_Shell[0]-Radius_Cell-Membrane_Cell-Glow_Cell) + center[0],
                       random.randint(-Radius_Shell[1]+Radius_Cell+Membrane_Cell+Glow_Cell,
                                       Radius_Shell[1]-Radius_Cell-Membrane_Cell-Glow_Cell) + center[1],
                       random.randint(-Radius_Shell[2]+Radius_Cell+Membrane_Cell+Glow_Cell,
                                       Radius_Shell[2]-Radius_Cell-Membrane_Cell-Glow_Cell) + center[2]])
    isvalid = ((Cell_Centers[Cell_Numbers,0]-center[0])**2 / (Radius_Shell[0]-Radius_Cell-Membrane_Cell-Glow_Cell-1)**2 + \
               (Cell_Centers[Cell_Numbers,1]-center[1])**2 / (Radius_Shell[1]-Radius_Cell-Membrane_Cell-Glow_Cell-1)**2 + \
               (Cell_Centers[Cell_Numbers,2]-center[2])**2 / (Radius_Shell[2]-Radius_Cell-Membrane_Cell-Glow_Cell-1)**2 <= 1) and \
              ((Cell_Centers[Cell_Numbers,0]-center[0])**2 / Radius_Interior[0]**2 + \
               (Cell_Centers[Cell_Numbers,1]-center[1])**2 / Radius_Interior[1]**2 + \
               (Cell_Centers[Cell_Numbers,2]-center[2])**2 / Radius_Interior[2]**2 > 1)
    for i in range(Cell_Numbers-1):
        if np.linalg.norm(Cell_Centers[Cell_Numbers,:]-Cell_Centers[i,:]) < 2*(Radius_Cell+Membrane_Cell):
            isvalid = False   
            break
    if isvalid:
        n = Cell_Numbers
        # Set Classes accordingly
        for h in range(         max(int(Cell_Centers[n,0]-Radius_Cell-Membrane_Cell-Glow_Cell),0),\
                                min(int(Cell_Centers[n,0]+Radius_Cell+Membrane_Cell+Glow_Cell),size[0])):
            for j in range(     max(int(Cell_Centers[n,1]-Radius_Cell-Membrane_Cell-Glow_Cell),0),\
                                min(int(Cell_Centers[n,1]+Radius_Cell+Membrane_Cell+Glow_Cell),size[1])):
                for l in range( max(int(Cell_Centers[n,2]-Radius_Cell-Membrane_Cell-Glow_Cell),0),\
                                min(int(Cell_Centers[n,2]+Radius_Cell+Membrane_Cell+Glow_Cell),size[2])):
                    dist = np.linalg.norm(Cell_Centers[n,:]-np.array([h,j,l]))
                    if dist <= Radius_Cell: # Interior
                        Geometry_Classes[h,j,l] = 1
                    elif dist <= (Radius_Cell+Membrane_Cell): # Membrane
                        Geometry_Classes[h,j,l] = 2
                    elif dist <= (Radius_Cell+Membrane_Cell+Glow_Cell): # GLow area
                        Geometry_Classes[h,j,l] = 3
        Cell_Numbers += 1
        Rejected = 0
    else:
        Rejected += 1
# Sampling of Interior complex
Radius_Nucleus   = 25
Membrane_Nucleus = 0
Glow_Nucleus     = 1
while True:
    Interior_Center = np.array(
                      [random.randint(-Radius_Interior[0]+Radius_Nucleus+Membrane_Nucleus+1,
                                       Radius_Interior[0]-Radius_Nucleus-Membrane_Nucleus-1)//2 + center[0],
                       random.randint(-Radius_Interior[1]+Radius_Nucleus+Membrane_Nucleus+1,
                                       Radius_Interior[1]-Radius_Nucleus-Membrane_Nucleus-1)//2 + center[1],
                       random.randint(-Radius_Interior[2]+Radius_Nucleus+Membrane_Nucleus+1,
                                       Radius_Interior[2]-Radius_Nucleus-Membrane_Nucleus-1)//2 + center[2]])
    if (Interior_Center[0]-center[0])**2 / Radius_Interior[0]**2 + \
       (Interior_Center[1]-center[1])**2 / Radius_Interior[1]**2 + \
       (Interior_Center[2]-center[2])**2 / Radius_Interior[2]**2 <=1:
        for h in range(         max(int(Interior_Center[0]-Radius_Nucleus-Membrane_Nucleus-Separator_Cell),0),\
                                min(int(Interior_Center[0]+Radius_Nucleus+Membrane_Nucleus+Separator_Cell),size[0])):
            for j in range(     max(int(Interior_Center[1]-Radius_Nucleus-Membrane_Nucleus-Separator_Cell),0),\
                                min(int(Interior_Center[1]+Radius_Nucleus+Membrane_Nucleus+Separator_Cell),size[1])):
                for l in range( max(int(Interior_Center[2]-Radius_Nucleus-Membrane_Nucleus-Separator_Cell),0),\
                                min(int(Interior_Center[2]+Radius_Nucleus+Membrane_Nucleus+Separator_Cell),size[2])):
                    dist = np.linalg.norm(Interior_Center-np.array([h,j,l]))
                    if dist <= Radius_Nucleus: # Interior Cell
                        Geometry_Classes[h,j,l] = 4
                    elif dist <= (Radius_Nucleus+Membrane_Nucleus): # Membrane
                        Geometry_Classes[h,j,l] = 2
                    elif dist <= (Radius_Nucleus+Membrane_Nucleus+Glow_Nucleus): # GLow area
                        Geometry_Classes[h,j,l] = 3
        break

        
# Reduce array correctly
Cell_Centers = Cell_Centers[np.arange(Cell_Numbers),:]   
fig, ax = plt.subplots(2,4,figsize=(20, 6))
ax[0,0].imshow( OuterGlow_Mask[:,:,size[2]//2],cmap='gray')
ax[0,1].imshow( OuterShell_Mask[:,:,size[2]//2],cmap='gray')
ax[0,2].imshow( InnerShell_Mask[:,:,size[2]//2],cmap='gray')
ax[0,3].imshow( OuterGlow_Mask[:,:,size[2]//2] + 
                OuterShell_Mask[:,:,size[2]//2] + 
                InnerShell_Mask[:,:,size[2]//2],cmap='gray')
colors = ['black', 'green', 'blue', 'red','yellow','cyan']
cmap = ListedColormap(colors)
ax[1,0].imshow( Geometry_Classes[:,:,96]  ,cmap=cmap)
ax[1,1].imshow( Geometry_Classes[:,:,128] ,cmap=cmap)
ax[1,2].imshow( Geometry_Classes[:,:,156],cmap=cmap)
ax[1,3].imshow( (Geometry_Classes + OuterShell_Mask*1)[:,:,128] ,cmap=cmap)


In [ ]:
%%capture
# Refractive indices & fluorophore distribution
# Perlin noises
rid_scale          = 2
fld_scale          = 10
rid_structure      = np.clip(perlin3(size = tuple(size)[::-1], scale = (rid_scale,rid_scale,rid_scale)),-1,1)
fld_structure      = np.clip(perlin3(size = tuple(size)[::-1], scale = (fld_scale,fld_scale,fld_scale)),-1,1)

# Refractive indices
rid_water          = 1.333 # Background
rid_cytosol        = np.array([1.36,1.39])
rid_nucleus        = np.array([1.355,1.365])
rid_nucleolus      = np.array([1.375,1.385])
rid_mitochondria   = np.array([1.4,1.42])
rid_lysosome       = np.array([1.6,1.6])
rid_membrane       = np.array([1.45,1.48])
# Constant indices - if preferred  
#rid_cytosol        = np.array([1.375,1.375])
#rid_nucleus        = np.array([1.36,1.36])
#rid_nucleolus      = np.array([1.38,1.38])
#rid_mitochondria   = np.array([1.41,1.41])
#rid_lysosome       = np.array([1.6,1.6])
#rid_membrane       = np.array([1.465,1.465])

refractive_indices = scipy.ndimage.gaussian_filter((np.mean(rid_nucleolus) - rid_water + (rid_nucleolus[1] - np.mean(rid_nucleolus))*rid_structure)*OuterGlow_Mask,sigma=3.)*(OuterShell_Mask==0)\
                   + (np.mean(rid_cytosol) - rid_water + (rid_cytosol[1] - np.mean(rid_cytosol)) * rid_structure) * np.logical_or(np.logical_and(Geometry_Classes == 0,OuterShell_Mask == 1),Geometry_Classes == 3) \
                   + (np.mean(rid_nucleus) - rid_water + (rid_nucleus[1] - np.mean(rid_nucleus)) * rid_structure) * (Geometry_Classes == 1) \
                   + (np.mean(rid_cytosol) - rid_water + (rid_cytosol[1] - np.mean(rid_cytosol)) * rid_structure) * (Geometry_Classes == 2) \
                   + (np.mean(rid_nucleolus) - rid_water + (rid_nucleolus[1] - np.mean(rid_nucleolus))*rid_structure) * (Geometry_Classes == 4)


# Fluorophore distribution
fld_water     = 0.25 # Background
fld_cytosol   = 0.65 # Filling constituent 
fld_membrane  = 0.75 # Outer membrane
fld_nucleus   = 1    # Cell nuclei
fld_nucleolus = 0.85
fld_variation = 0.15 # "Natural" variation in fluorescence
# Alternative Nulceolus
fld_nucleolus = fld_cytosol

fluorescence  = (fld_water     + fld_variation * fld_water    * fld_structure) * np.logical_and(OuterShell_Mask == 0, Geometry_Classes == 0) \
              + (1-fld_variation)*scipy.ndimage.gaussian_filter((fld_membrane - fld_water)*OuterGlow_Mask,sigma=3.)*(OuterShell_Mask == 0) \
              + (fld_cytosol   + fld_variation * fld_cytosol  * fld_structure) * np.logical_or(np.logical_and(OuterShell_Mask == 1, Geometry_Classes == 0 ), Geometry_Classes == 3) \
              + (fld_nucleus   + fld_variation * fld_nucleus  * rid_structure) * (Geometry_Classes == 1) \
              + (fld_membrane  + fld_variation * fld_membrane * fld_structure) * (Geometry_Classes == 2) \
              + (fld_nucleolus + fld_variation * fld_nucleolus* fld_structure) * (Geometry_Classes == 4)

fig, ax = plt.subplots(2,3,figsize=(20, 6))
ax[0,0].imshow( refractive_indices[:,:,96] ,cmap='gray')
ax[0,1].imshow( refractive_indices[:,:,128] ,cmap='gray')
ax[0,2].imshow( refractive_indices[:,:,156],cmap='gray')
ax[1,0].imshow( fluorescence[:,:,96] ,cmap='gray')
ax[1,1].imshow( fluorescence[:,:,128],cmap='gray')
ax[1,2].imshow( fluorescence[:,:,156],cmap='gray')

In [ ]:
%%capture

# directions in deg (array)
angle = [0] # default [0], bi-directional e.g. [-5,5]
            # multidirectional e.g. [-5,-4,-3,-2,-1,0,1,2,3,4,5]

# Light wavelength
lmbd = 0.5 # default: 0.5

# Aperture sizes for detection and the cylindrical light sheets
NAdet = 0.5 # default: 0.95
NAls = 0.1 # default: 0.1

# Sampling in z-direction 
zsample = 1
# Focal plane distance
z_foc = .5*size[2]

# (efficiency) Set a maximum kernel size/height for the detection PSF:
kerint = 64 

# Create geometry 
# Space of each pixel in mikrons (mikrometer)
units = (0.4,)*3 # (.2)
m = biobeam.Bpm3d(dn = refractive_indices.transpose(2,0,1), units = units, lam = lmbd) 
m_ref = biobeam.Bpm3d(dn = np.zeros(size).transpose(2,0,1), units = units, lam = lmbd)  

# Create detection beam
v = np.zeros([min(size[0],kerint),size[1],size[2]])
v[:,size[1]//2-min(size[1]//2,kerint//2):size[1]//2+min(size[1]//2,kerint//2),
  size[2]//2-min(size[2]//2,kerint//2):size[2]//2+min(size[2]//2,kerint//2)] = biobeam.focus_field_beam(shape = [min(size[2],kerint),min(size[1],kerint),min(size[0],kerint)],
                                                                                                  units = units,NA = NAdet, n0 = 1.33)
# Illumination results (initialization)
res = np.zeros(((size[0]-kerint)//zsample,size[1],size[2]))
ref = np.zeros(((size[0]-kerint)//zsample,size[1],size[2]))

# Light propagation simulation:
for k in range((size[0]-kerint)//zsample): # iterate over all imaged slices
    # illumination response
    u = biobeam.focus_field_cylindrical_plane(shape=(size[1],size[0]),units = units[0:2],NA = NAls, lam = lmbd,z = z_foc,n0 = rid_water);
    u_ref = m_ref.propagate(u0 = np.roll(u,zsample*k-size[0]//2+kerint//2,0), return_comp = "intens");
    u     =     m.propagate(u0 = np.roll(u,zsample*k-size[0]//2+kerint//2,0), return_comp = "intens");
    u     = np.multiply(u[:,zsample*k:zsample*k+kerint,:],fluorescence[zsample*k:zsample*k+kerint,:,:].transpose(2,0,1)).transpose(1,2,0)
    u     = fft_convolve(u,v)
    u_ref = np.multiply(u_ref[:,zsample*k:zsample*k+kerint,:],fluorescence[zsample*k:zsample*k+kerint,:,:].transpose(2,0,1)).transpose(1,2,0)
    u_ref = fft_convolve(u_ref,v)
    res[k,:,:] = u[kerint//2,:,:]
    ref[k,:,:] = u_ref[kerint//2,:,:] 
imsave(os.getcwd() + '/Synthetic-Embryo-flat.tif',res)
imsave(os.getcwd() + '/Synthetic-Embryo-flat_Ideal.tif',ref)

In [ ]:
fig, ax = plt.subplots(4,3,figsize=(20, 15))
ax[0,0].imshow( refractive_indices[:,:,96] ,cmap='gray')
ax[0,1].imshow( refractive_indices[:,:,128] ,cmap='gray')
ax[0,2].imshow( refractive_indices[:,:,156],cmap='gray')
ax[1,0].imshow( fluorescence[:,:,96] ,cmap='gray')
ax[1,1].imshow( fluorescence[:,:,128],cmap='gray')
ax[1,2].imshow( fluorescence[:,:,156],cmap='gray')
ax[2,0].imshow( res[96-kerint//2,:,:].transpose() ,cmap='gray')
ax[2,1].imshow( res[128-kerint//2,:,:].transpose(),cmap='gray')
ax[2,2].imshow( res[156-kerint//2,:,:].transpose(),cmap='gray')
ax[3,0].imshow( ref[96-kerint//2,:,:].transpose() ,cmap='gray')
ax[3,1].imshow( ref[128-kerint//2,:,:].transpose(),cmap='gray')
ax[3,2].imshow( ref[156-kerint//2,:,:].transpose(),cmap='gray')
fig.tight_layout()